In [1]:
import pandas as pd
import numpy as np
import cv2

In [20]:
common_path ="subject1/"

In [21]:
# Function to map gaze positions to pixel coordinates
def get_row_number(eye_tracking_data, des_gidx):
    if des_gidx in eye_tracking_data['gidx'].values:
        row_number = eye_tracking_data.index[eye_tracking_data['gidx'] == des_gidx].tolist()
        print('Exist')
        return row_number[0] if row_number else None
    else:
        return None
    
def map_gaze_to_pixels(gaze_position_x,gaze_position_y, frame_width, frame_height):
    if gaze_position_x is not None and gaze_position_y is not None:
        mapped_x = int(gaze_position_x * frame_width)
        mapped_y = int(gaze_position_y * frame_height)
        return mapped_x, mapped_y
    return None

def overlay_gaze_on_frame(frame, des_row,eye_tracking_data,frame_width,frame_height):
    last_row_number = eye_tracking_data.shape[0] - 1
    # print(des_row)
    for i in range(4):
        if (des_row+i) <= last_row_number:
            gaze_position_x = eye_tracking_data['gaze_position_x'].iloc[des_row+i]
            gaze_position_y = eye_tracking_data['gaze_position_y'].iloc[des_row+i]
            print(f'row:{des_row}')

            # Map gaze point to pixel coordinates
            mapped_gaze_position = map_gaze_to_pixels(gaze_position_x,gaze_position_y, frame_width, frame_height)
            if mapped_gaze_position is not None:
                x, y = mapped_gaze_position
                if x !=0 and y!=0:
                    cv2.circle(frame, (int(x), int(y)), 10, (0, 0, 255), -1)  # Red color, larger radius
        else:
            break
    return frame

### Plotting gaze positions

In [4]:

# Video file path
video_file_path = f'{common_path}fullstream.mp4'
eye_tracking_file_path = f'{common_path}merged_data.csv'  # Replace with the actual gaze data file

# Read video frames
cap = cv2.VideoCapture(video_file_path)
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
else:
    # Read gaze data
    eye_tracking_data = pd.read_csv(eye_tracking_file_path,delimiter=',')
    # Get video details
    width  = int(cap.get(3))
    height = int(cap.get(4))
    fps    = cap.get(5)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{common_path}gaze_mapping_video.mp4', fourcc, fps, (width, height))

    first_gidx = eye_tracking_data['gidx'].iloc[0]
    # Process video frames
    for frame_count in range(int(cap.get(7))):  # Total number of frames
        ret, frame = cap.read()

        # Break the loop if the video has ended
        if not ret:
            break

        # Check if gaze data is available for the current frame
        des_row = get_row_number(eye_tracking_data,(frame_count*4 +first_gidx))  # Assuming frame_count starts from 1
        if des_row is not None:
            # Overlay mapped gaze point on the frame
            frame_with_gaze = overlay_gaze_on_frame(frame.copy(),des_row,eye_tracking_data,width,height)

        # Write the frame to the output video
        out.write(frame_with_gaze)

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Close all windows
    cv2.destroyAllWindows()

    print("Output video saved as 'gaze_mapping_video.mp4'")

Exist
row:0
row:0
row:0
row:0
Exist
row:5
row:5
row:5
row:5
Exist
row:20
row:20
row:20
row:20
Exist
row:25
row:25
row:25
row:25
Exist
row:29
row:29
row:29
row:29
Exist
row:33
row:33
row:33
row:33
Exist
row:37
row:37
row:37
row:37
Exist
row:41
row:41
row:41
row:41
Exist
row:45
row:45
row:45
row:45
Exist
row:49
row:49
row:49
row:49
Exist
row:53
row:53
row:53
row:53
Exist
row:57
row:57
row:57
row:57
Exist
row:61
row:61
row:61
row:61
Exist
row:65
row:65
row:65
row:65
Exist
row:69
row:69
row:69
row:69
Exist
row:73
row:73
row:73
row:73
Exist
row:77
row:77
row:77
row:77
Exist
row:81
row:81
row:81
row:81
Exist
row:85
row:85
row:85
row:85
Exist
row:89
row:89
row:89
row:89
Exist
row:93
row:93
row:93
row:93
Exist
row:97
row:97
row:97
row:97
Exist
row:101
row:101
row:101
row:101
Exist
row:105
row:105
row:105
row:105
Exist
row:109
row:109
row:109
row:109
Exist
row:113
row:113
row:113
row:113
Exist
row:117
row:117
row:117
row:117
Exist
row:121
row:121
row:121
row:121
Exist
row:125
row:125
row:125
ro

### Plotting gaze positions (color coded)
##### Red - Fixation
##### Green - Saccade
##### Blue - Unclassified

In [23]:
# Function to map gaze positions to pixel coordinates
def get_row_number(eye_tracking_data, des_gidx):
    if des_gidx in eye_tracking_data['gidx'].values:
        row_number = eye_tracking_data.index[eye_tracking_data['gidx'] == des_gidx].tolist()
        return row_number[0] if row_number else None
    else:
        return None

def map_gaze_to_pixels(gaze_position_x, gaze_position_y, frame_width, frame_height):
    if gaze_position_x is not None and gaze_position_y is not None:
        mapped_x = int(gaze_position_x * frame_width)
        mapped_y = int(gaze_position_y * frame_height)
        return mapped_x, mapped_y
    return None

def overlay_gaze_on_frame(frame, des_row, eye_tracking_data, frame_width, frame_height):
    last_row_number = eye_tracking_data.shape[0] - 1
    for i in range(4):
        if (des_row+i) <= last_row_number:
            gaze_position_x = eye_tracking_data['gaze_position_x'].iloc[des_row+i]
            gaze_position_y = eye_tracking_data['gaze_position_y'].iloc[des_row+i]
            eye_movement_type = eye_tracking_data['Eye movement type'].iloc[des_row+i]

            # Map gaze point to pixel coordinates
            mapped_gaze_position = map_gaze_to_pixels(gaze_position_x, gaze_position_y, frame_width, frame_height)
            if mapped_gaze_position is not None:
                x, y = mapped_gaze_position
                if x != 0 and y != 0:
                    if eye_movement_type == 'Saccade':
                        cv2.circle(frame, (int(x), int(y)), 10, (0, 255, 0), -1)  # Green for saccade
                    elif eye_movement_type == 'Fixation':
                        cv2.circle(frame, (int(x), int(y)), 10, (0, 0, 255), -1)  # Red for fixation
                    else:
                        cv2.circle(frame, (int(x), int(y)), 10, (255, 0, 0), -1)  # Blue for other movements

                    # Draw vectors between consecutive gaze positions
                    if i > 0:
                        prev_gaze_position_x = eye_tracking_data['gaze_position_x'].iloc[des_row+i-1]
                        prev_gaze_position_y = eye_tracking_data['gaze_position_y'].iloc[des_row+i-1]
                        prev_mapped_gaze_position = map_gaze_to_pixels(prev_gaze_position_x, prev_gaze_position_y, frame_width, frame_height)
                        if prev_mapped_gaze_position is not None:
                            prev_x, prev_y = prev_mapped_gaze_position
                            cv2.arrowedLine(frame, (prev_x, prev_y), (x, y), (0, 255, 0), 2)  # Green for vectors

        else:
            break
    return frame

# Video file path
video_file_path = f'{common_path}fullstream.mp4'  # Modify with your video file path
eye_tracking_file_path = f'{common_path}merged_data.csv'  # Modify with your gaze data file path

# Read video frames
cap = cv2.VideoCapture(video_file_path)
if not cap.isOpened():
    print("Error: Couldn't open the video file.")
else:
    # Read gaze data
    eye_tracking_data = pd.read_csv(eye_tracking_file_path)

    # Get video details
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{common_path}gaze_mapping_video.mp4', fourcc, fps, (width, height))

    first_gidx = eye_tracking_data['gidx'].iloc[0]

    # Process video frames
    for frame_count in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):  # Total number of frames
        ret, frame = cap.read()

        # Break the loop if the video has ended
        if not ret:
            break

        # Check if gaze data is available for the current frame
        des_row = get_row_number(eye_tracking_data, (frame_count * 4 + first_gidx))  # Assuming frame_count starts from 1
        if des_row is not None:
            # Overlay mapped gaze point on the frame
            frame_with_gaze = overlay_gaze_on_frame(frame.copy(), des_row, eye_tracking_data, width, height)
        else:
            frame_with_gaze = frame

        # Write the frame to the output video
        out.write(frame_with_gaze)

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Close all windows
    cv2.destroyAllWindows()

    print("Output video saved as 'gaze_mapping_video.mp4'")


Output video saved as 'gaze_mapping_video.mp4'
